In [2]:
import pandas as pd
import os
import sys
import inspect
import s2_tst as s2

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
neutral_messages = pd.read_csv('00_input_data/messages.csv')
neutral_messages

,messageID,message
0,0,"Hello! I’m Sasha, nice to meet you! Over the n..."
1,1,Hey! Thanks for following along for the first ...
2,2,Reducing how much meat we eat can help save a ...
3,3,"If everyone ate less animal foods, the CO2 fro..."
4,4,Animal farming is responsible for 80% of the l...
5,5,80% of young people believe that living sustai...
6,6,"Reducing how much meat, dairy, and sugary food..."
7,7,Fruits and vegetables are filled with importan...
8,8,Fiber is really important for keeping our stom...
9,9,Vegetables and fruits have helpful plant subst...


In [80]:
## TST 

prompt = """
Task: You are an expert in text style transfer. You will be provided with example sentences showcasing the conversational style of person X, along with corresponding sentences written without any style.
Your task is to learn the conversational style of person X and rewrite a target text written without style in person X’s conversational style.

Constraints:
1. The rewritten target text must not deviate more than ±25% in length from the original target text.
2. To replicate person X’s style, consider how they use function words (e.g., pronouns like "I", "we", "you"), their preferred tone (e.g., formality, analytical, or narrative), and other stylistic traits (e.g., emotional tone, complexity, common phrases/fillers, punctuation).
3. Only include phrases, fillers, emoticons, or emojis explicitly used by person X in the example sentences.

Here are example sentences without style: {}
Here are example sentences rewritten in the style of person X: {}

Here is the target text without any style: {}

Result format in json:
"rewritten_sentence": "",
"explanation": ""
"""


prompt2 = """
Task: You are an expert in text style transfer. You will be provided with example sentences showcasing the conversational style of person X, along with corresponding sentences written without any style.
Your task is to learn the conversational style of person X and rewrite a target text written without style in person X’s conversational style.

Constraints:

1. The rewritten target text must not deviate more than ±25% in length from the original target text.
2. To replicate person X’s style, consider how they use function words (e.g., pronouns like "I", "we", "you"), their preferred tone (e.g., formality, analytical, or narrative), and other stylistic traits (e.g., emotional tone, complexity, common phrases/fillers, punctuation).
3. Only include phrases, fillers, emoticons, or emojis explicitly used by person X in the example sentences.
4. Don't use any variation of 'haha' in inappropriate places.

Here are example sentences without style: {}
Here are example sentences rewritten in the style of person X: {}

Here is the target text without any style: {}

Result format in json:
"rewritten_sentence": "",
"explanation": ""
"""



## TST Mistral

In [81]:
# 'open-mistral-nemo', 'mistral-large-latest','gpt-4o-mini','gpt-4o'
model = 'mistral-large-latest'

In [82]:
# all shots
examples = pd.read_csv('01_processed_input_data/all_shots_data_mistral.csv')
input_data = s2.get_df_tst(examples)


100%|██████████| 8/8 [00:00<00:00, 2152.17it/s]


In [84]:
# prompt 1
tst_mistral = s2.llm_tst_exploration(input_data.iloc[[2,3,4]], neutral_messages, model, prompt, 'all')
tst_mistral.to_csv('02_tst_exploration/'+model+'/'+'parallel_all_shots_exploration.csv')

Processing LLM TST evaluation...: 100%|██████████| 3/3 [04:23<00:00, 87.68s/it]


In [85]:
# prompt 2
tst_mistral = s2.llm_tst_exploration(input_data.iloc[[2,3,4]], neutral_messages, model, prompt2, 'all')
tst_mistral.to_csv('02_tst_exploration/'+model+'/'+'parallel_all_shots_exploration_strict.csv')

Processing LLM TST evaluation...: 100%|██████████| 3/3 [04:11<00:00, 83.84s/it]


### postprocess tst output


In [86]:
tst_mistral = pd.read_csv("02_tst_exploration/mistral-large-latest/parallel_all_shots_exploration.csv")
df = s2.parse_tst_data(tst_mistral)
df_temp = df[['promptID', 'model','shots', 'username', 'tst_id', 'message_id', 'neutral_message', 'tst_message', 'tst_explanation' ]]
df_temp.to_csv('02_tst_exploration/mistral-large-latest/parallel_all_shots_exploration_processed.csv')

wronglyParsed:  0


In [87]:
tst_mistral = pd.read_csv("02_tst_exploration/mistral-large-latest/parallel_all_shots_exploration_strict.csv")
df = s2.parse_tst_data(tst_mistral)
df_temp = df[['promptID', 'model','shots', 'username', 'tst_id', 'message_id', 'neutral_message', 'tst_message', 'tst_explanation' ]]
df_temp.to_csv('02_tst_exploration/mistral-large-latest/parallel_all_shots_exploration_strict_processed.csv')

wronglyParsed:  0


## TST GPT

In [63]:
# 'open-mistral-nemo', 'mistral-large-latest','gpt-4o-mini','gpt-4o'
model = 'gpt-4o'

In [59]:
# all shots
examples = pd.read_csv('01_processed_input_data/all_shots_data_gpt.csv')
input_data = s2.get_df_tst(examples)


100%|██████████| 8/8 [00:00<00:00, 3032.76it/s]


In [60]:
# prompt 1
tst_gpt = s2.llm_tst_exploration(input_data.iloc[[2,4]], neutral_messages, model, prompt, 'all')
tst_gpt.to_csv('02_tst_exploration/'+model+'/'+'parallel_all_shots_exploration.csv')

Processing LLM TST evaluation...: 100%|██████████| 2/2 [02:07<00:00, 63.83s/it]


In [32]:
# prompt 2
tst_gpt = s2.llm_tst_exploration(input_data.iloc[[2,4]], neutral_messages, model, prompt2, 'all')
tst_gpt.to_csv('02_tst_exploration/'+model+'/'+'parallel_all_shots_exploration_strict.csv')

Generating LLM TST Messages per Participant...: 100%|██████████| 8/8 [03:34<00:00, 26.85s/it]


### postprocess tst output

In [64]:
tst_gpt = pd.read_csv("02_tst_exploration/gpt-4o/parallel_all_shots_exploration.csv")
df = s2.parse_tst_data(tst_gpt)
df_temp = df[['promptID', 'model','shots', 'username', 'tst_id', 'message_id', 'neutral_message', 'tst_message', 'tst_explanation' ]]
df_temp.to_csv('02_tst_exploration/gpt-4o/parallel_all_shots_exploration_processed.csv')

wronglyParsed:  0


In [65]:
tst_gpt = pd.read_csv("02_tst_exploration/gpt-4o/parallel_all_shots_exploration_strict.csv")
df = s2.parse_tst_data(tst_gpt)
df_temp = df[['promptID', 'model','shots', 'username', 'tst_id', 'message_id', 'neutral_message', 'tst_message', 'tst_explanation' ]]
df_temp.to_csv('02_tst_exploration/gpt-4o/parallel_all_shots_exploration_strict_processed.csv')

wronglyParsed:  0
